In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/1500-unsplash-images-metadata/1500 Unsplash Images Metadata.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import chi2_contingency, f_oneway
import statsmodels.api as sm

In [3]:
df = pd.read_csv("/kaggle/input/1500-unsplash-images-metadata/1500 Unsplash Images Metadata.csv")
df.head()

,id,width,height,color,downloads,likes,make,model,exposure_time,aperture,focal_length,iso,city,country
0,uI3CBm8udb4,2345,2931,#264040,4287,118,HUAWEI,VOG-L29,1/1000,3.4,14.5,100,Unknown,Portugal
1,Mbd2gOxxK5I,3351,4189,#EFEFEF,2511,28,SONY,ILCE-7M3,Unknown,1.4,85,100,Unknown,Unknown
2,im3Zz7gZGUI,4575,6100,#404026,5225,118,DJI,FC3582,Unknown,1.7,6.7,100,Unknown,Indonesia
3,Y1G_0YYYA3c,2812,4455,#262626,4423,64,FUJIFILM,X-T1,Unknown,1,50,200,Unknown,Unknown
4,GAu0pSfgqNc,5104,2871,#5973a6,5222,80,Canon,EOS R6,Unknown,20,400,200,Unknown,Spain


In [4]:
df.shape

(1539, 14)

In [5]:
df.describe()

,width,height,downloads,likes
count,1539.000000,1539.000000,1539.000000,1539.000000
mean,4675.552956,4984.333333,4932.345679,62.160494
std,1670.269691,1626.932274,12159.017930,101.706062
min,1828.000000,1415.000000,0.000000,2.000000
25%,3652.500000,3873.000000,2123.500000,28.000000
50%,4160.000000,5015.000000,3140.000000,47.000000
75%,5504.000000,6000.000000,4728.500000,72.000000
max,16384.000000,14999.000000,326016.000000,2847.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1539 non-null   object
 1   width          1539 non-null   int64 
 2   height         1539 non-null   int64 
 3   color          1539 non-null   object
 4   downloads      1539 non-null   int64 
 5   likes          1539 non-null   int64 
 6   make           1539 non-null   object
 7   model          1539 non-null   object
 8   exposure_time  1539 non-null   object
 9   aperture       1539 non-null   object
 10  focal_length   1539 non-null   object
 11  iso            1539 non-null   object
 12  city           1539 non-null   object
 13  country        1539 non-null   object
dtypes: int64(4), object(10)
memory usage: 168.5+ KB


In [7]:
# Random Sampling
random_sample = df.sample(frac=0.1)['width']
random_sample

196     5005
1531    5091
174     5184
904     4000
1363    3276
        ... 
644     4000
1322    5567
490     4565
337     5600
1195    5827
Name: width, Length: 154, dtype: int64

In [8]:
# Systematic sampling

# Calculate the interval as an integer
interval = int(len(df) // (len(df) * 0.1))

# Randomly choose a starting point within the interval
start_point = np.random.randint(0, interval)

# Perform systematic sampling
systematic_sample = df.iloc[start_point::interval]['width']

systematic_sample

2       4575
11      4884
20      2832
29      7621
38      6240
        ... 
1496    3866
1505    4094
1514    3181
1523    2201
1532    2500
Name: width, Length: 171, dtype: int64

In [9]:
# First DataFrame: 'make' and total count in the whole population
make_counts_df = df['make'].value_counts().reset_index()
make_counts_df.columns = ['make', 'count']

# Second DataFrame: 'make' and count in the Stratified Sampling
# Note: 'stratified_sample' only contains 'width' values. We need to work with the sampled DataFrame, not the 'width' Series.
stratified_sample_df = df.groupby('make', group_keys=False).apply(lambda x: x.sample(frac=0.1))
make_counts_sampled_df = stratified_sample_df['make'].value_counts().reset_index()
make_counts_sampled_df.columns = ['make', 'count']

print(make_counts_df.head(7))
print("***************************")
print(make_counts_sampled_df.head(8))

                make  count
0            Unknown    434
1               SONY    356
2              Canon    290
3           FUJIFILM    145
4  NIKON CORPORATION    129
5                DJI     35
6          Panasonic     24
***************************
                make  count
0            Unknown     43
1               SONY     36
2              Canon     29
3           FUJIFILM     14
4  NIKON CORPORATION     13
5                DJI      4
6              Apple      2
7         Hasselblad      2


#### 1-sample Z test for mean
1-sample Z test for mean: This test is used to determine whether the sample mean of the data is significantly different from a known or hypothesized population mean. It's typically used when the population standard deviation is known and the sample size is large (n > 30).

In [10]:
from scipy import stats

# Random Sampling
random_sample = df.sample(frac=0.1)['width']

# Hypothetical population mean and standard deviation
hypothetical_mean = 4675  # Hypothetical population mean
population_std = 1670  # Actual population standard deviation

# Calculating sample statistics for the random sample
random_sample_mean = random_sample.mean()
n = len(random_sample)  # Corrected to use the size of the random sample

# 1-sample Z test calculation
z_score = (random_sample_mean - hypothetical_mean) / (population_std / np.sqrt(n))
p_value = stats.norm.sf(abs(z_score)) * 2  # Two-tailed test

print(f"Z score = {z_score}, P-value = {p_value}")

Z score = 0.6585069455122395, P-value = 0.5102124355063016


##### Result
In 1-sample Z test, the null hypothesis (H0) states that the mean width of the dataset's sample equals the hypothetical population mean (4675), indicating no significant difference. The alternative hypothesis (H1) posits that the sample mean differs significantly from the hypothetical mean. The choice between these hypotheses depends on the p-value compared to a pre-set significance level (typically 0.05). A p-value less than or equal to the significance level suggests rejecting H0 in favor of H1, indicating a significant difference. Conversely, a p-value greater than the significance level suggests insufficient evidence to reject H0, meaning any observed difference could be due to random chance. With a p-value of approximately 0.783, significantly above 0.05, you fail to reject the null hypothesis. This result indicates insufficient evidence to conclude a significant difference between the sample mean width and the hypothetical population mean, suggesting the observed difference is likely due to random variation.

#### 1-sample T test for mean
1-sample T test for mean: This test is similar to the Z test but is used when the population standard deviation is unknown and the sample size is small (n < 30). It uses the sample standard deviation as an estimate of the population standard deviation.

In [11]:
import pandas as pd
from scipy import stats

# Random Sampling
random_sample = df.sample(frac=0.017)['width']

# Hypothetical population mean
hypothetical_mean = 4675  # Hypothetical population mean

# Performing 1-sample T test
t_score, p_value = stats.ttest_1samp(random_sample, hypothetical_mean)

print(f"T score = {t_score}, P-value = {p_value}")

T score = 1.1229561094357143, P-value = 0.2721250503163452


##### Result
In 1-sample T test, the null hypothesis (H0) posits that the mean width of the dataset's sample is equal to the hypothetical population mean of 4675, implying no significant difference. The alternative hypothesis (H1) asserts that the sample mean significantly differs from the hypothetical mean. The decision between H0 and H1 is based on the p-value in relation to a chosen significance level, typically 0.05. A p-value ≤ 0.05 suggests rejecting H0 in favor of H1, indicating a statistically significant difference. Conversely, a p-value > 0.05 indicates insufficient evidence to reject H0, suggesting that any observed difference might be due to chance. With a p-value of approximately 0.367, you fail to reject the null hypothesis. This outcome suggests there's insufficient evidence to conclude a significant difference between the sample mean width and the hypothetical population mean, pointing towards the observed difference being likely due to random variation.

####  2 sample independent test for mean
Used in case of Controlled and Experimental group
This test is appropriate when you have two separate groups, and you want to determine if there is a statistically
significant difference between the means of these groups.
The data in each group should be approximately normally distributed.
The variances in the two groups should be roughly equal.
*When you want to compare the means of two independent groups, you can use a two-sample independent t-test.

In [12]:
from scipy import stats

# Creating 2 random samples from the 'width' column
sample1 = df['width'].sample(n=29)  
sample2 = df['width'].sample(n=29)  

# Performing 2-sample independent t-test
t_score, p_value = stats.ttest_ind(sample1, sample2)

print(f"T score = {t_score}, P-value = {p_value}")


T score = 1.8526886375769045, P-value = 0.06919945612263438


##### Result
Null Hypothesis (H0): The means of the two samples are equal.
Alternative Hypothesis (H1): The means of the two samples are not equal.

The 2-sample independent t-test yielded a T score of 1.8518 and a p-value of 0.0693. Since the p-value is greater than the typical significance level of 0.05, we fail to reject the Null Hypothesis (H0) that the means of the two samples are equal. This result suggests there is not enough statistical evidence to conclude a significant difference between the means of the two samples from the "width" column.

#### Matched/Paired sample test
The paired-sample t-test is commonly used in situations where you have two measurements taken on the same subjects or 
units, such as before-and-after measurements, repeated measurements on the same individuals, or matched pairs in a
study.

In [13]:
from scipy import stats

sample1 = df['width'].sample(n=29)  # Measurements under condition 1
sample2 = df['width'].sample(n=29)  # Corresponding measurements under condition 2

# Performing Paired sample t-test
t_score, p_value = stats.ttest_rel(sample1, sample2)

print(f"T score = {t_score}, P-value = {p_value}")

T score = 1.0190677346227324, P-value = 0.31689577755924936


#### Result
For Paired sample t-test, the null hypothesis (H0) states that the mean difference between the paired samples is zero, suggesting no significant effect between the two conditions. The alternative hypothesis (H1) posits that the mean difference is not zero, indicating a significant effect. With a T score of 1.0267 and a p-value of 0.3134, the p-value exceeds the typical significance level of 0.05. Therefore, you fail to reject the null hypothesis, suggesting insufficient evidence to conclude a significant difference between the paired samples. This decision is based on the statistical analysis indicating that the observed differences could likely be due to random chance rather than a systematic effect of the conditions. The outcome implies that, based on the samples tested, there's no strong evidence to support a change or effect between the two conditions under investigation.

#### Correlation
Correlation measures the strength and direction of a linear relationship between two quantitative variables. It is quantified by the correlation coefficient, which ranges from -1 to 1. A coefficient close to 1 indicates a strong positive correlation, meaning as one variable increases, the other tends to increase as well. A coefficient close to -1 indicates a strong negative correlation, meaning as one variable increases, the other tends to decrease. A coefficient around 0 suggests no linear correlation between the variables. Pearson’s correlation coefficient is the most common measure for linear relationships.

In [14]:
from scipy.stats import pearsonr

# Assuming 'df' is your DataFrame
correlation, p_value = pearsonr(df['width'], df['height'])

print(f"The Pearson correlation coefficient between width and height is: {correlation}")
print(f"The p-value for the correlation is: {p_value}")

The Pearson correlation coefficient between width and height is: 0.32153599620932816
The p-value for the correlation is: 2.379317842550679e-38


##### Result:
Null Hypothesis : There is no linear relationship between 'width' and 'height' (correlation coefficient = 0).

Alternative Hypothesis : There is a linear relationship between 'width' and 'height' (correlation coefficient ≠ 0).

Given the very low p-value, we reject the null hypothesis in favor of the alternative hypothesis. This means there is statistically significant evidence of a linear relationship between 'width' and 'height'. However, it's important to note that while the relationship is statistically significant, the moderate value of the correlation coefficient suggests that the strength of this relationship is not very strong.

#### Chi-square test

The Chi-square test is a statistical method used to determine the independence of two categorical variables. Here does 'make' depends on 'country'. By comparing observed frequencies in a contingency table with expected frequencies derived under the null hypothesis of independence, the test calculates a Chi-square statistic. This statistic measures the discrepancy between observed and expected frequencies. A p-value is then derived from the statistic, indicating the probability of observing the data, or more extreme, if the null hypothesis were true. A small p-value suggests rejecting the null hypothesis, indicating a significant association between the variables.

In [15]:
from scipy.stats import chi2_contingency

# Assuming df is your DataFrame and it contains 'make' and 'country' columns

# Creating a contingency table
contingency_table = pd.crosstab(df['make'], df['country'])

# Performing the Chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic = {chi2}, P-value = {p_value}")

Chi-square statistic = 4221.572743304037, P-value = 1.101378073467452e-37


##### Result

Null Hypothesis: There is no association between the 'make' of a product and the 'country' of origin; any observed difference in the contingency table is due to chance.

Alternative Hypothesis : There is an association between the 'make' of a product and the 'country' of origin; the observed difference in the contingency table is not due to chance.

Given the extremely small P-value, we reject the null hypothesis. This P-value indicates that the probability of observing the data or something more extreme if the null hypothesis were true is virtually zero. Therefore, we conclude that there is a statistically significant association between the 'make' of a product and the 'country' of origin. This means that the 'make' and 'country' variables are not independent, and the differences observed in the contingency table are not due to random chance.

#### 1-Way ANOVA
1-Way ANOVA (Analysis of Variance) is a statistical test used to determine if there are statistically significant differences between the means of three or more independent (unrelated) groups. It compares the variance within groups to the variance between groups to assess whether the group means differ beyond what could be expected by chance. The test calculates an F-statistic and a corresponding P-value, where a low P-value suggests that at least one group mean is significantly different from the others, indicating that the categorical variable has an effect on the continuous variable being studied.

In [16]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Perform 1-Way ANOVA to see if 'downloads' differs by 'make'
model_1way = ols('downloads ~ C(make)', data=df).fit()
anova_1way = sm.stats.anova_lm(model_1way, typ=2)

print(anova_1way)

                sum_sq      df         F   PR(>F)
C(make)   3.256612e+09    32.0  0.683837  0.90896
Residual  2.241239e+11  1506.0       NaN      NaN


##### Result
Null Hypothesis : There is no significant difference in the mean number of downloads across different makes. In other words, 'make' has no effect on 'downloads'.

Alternative Hypothesis : There is a significant difference in the mean number of downloads across different makes. This means 'make' affects 'downloads'.

The ANOVA test yielded an F-statistic of 0.683837 and a P-value of 0.90896. The high P-value suggests that we fail to reject the null hypothesis. This indicates that there is no statistically significant difference in the mean number of downloads for different makes of the product in your dataset. In other words, the variation in downloads cannot be attributed to the 'make' of the product, based on the data provided. This result underscores the importance of looking beyond just one factor when analyzing factors that influence downloads, as 'make' alone does not appear to be a determining factor.

#### 2-Way ANOVA
2-Way ANOVA (Analysis of Variance) is a statistical test used to examine the effects of two categorical independent variables on a continuous dependent variable, and to determine if there is an interaction between the two independent variables. It assesses whether the mean differences among and across the groups for these variables are statistically significant. By comparing the variance within groups to the variance between groups, 2-Way ANOVA can identify not only the individual effects of each independent variable but also whether the variables interact in a way that affects the dependent variable's outcomes.

In [17]:
# Perform 2-Way ANOVA to see if 'downloads' is affected by 'make' and 'model', and their interaction
model_2way = ols('downloads ~ C(make) + C(model) + C(make):C(model)', data=df).fit()
anova_2way = sm.stats.anova_lm(model_2way, typ=2)

print(anova_2way)

/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 32, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 186, but rank is 5
  warnings.warn('covariance of constraints does not have full '


                        sum_sq      df             F        PR(>F)
C(make)           9.976978e+03    32.0  2.948471e-06  9.986302e-01
C(model)         -2.422413e+02   186.0 -1.231638e-08  1.000000e+00
C(make):C(model)  2.719688e+12  5952.0  4.321196e+00  6.617507e-56
Residual          1.429647e+11  1352.0           NaN           NaN


/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5952, but rank is 185
  warnings.warn('covariance of constraints does not have full '


##### Result
Null Hypotheses

H01: There is no significant effect of 'make' on the number of downloads.

H02: There is no significant effect of 'model' on the number of downloads.

H03: There is no significant interaction effect between 'make' and 'model' on the number of downloads.

Alternative Hypotheses:

At least one of the null hypotheses is false, indicating that there is a significant effect of 'make', 'model', or their interaction on the number of downloads.

The ANOVA results show significant interaction effects between 'make' and 'model' on downloads, with an F-statistic of approximately 4.32 and a P-value of 6.61*10^-56. This suggests we reject H03, indicating a significant interaction effect between 'make' and 'model' on the number of downloads. However, the individual effects of 'make' and 'model' do not show statistical significance based on their P-values, suggesting that the impact on downloads is primarily through their interaction rather than their individual effects. This highlights the importance of considering how different factors can work together to influence outcomes.